In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts
from holoviews.element.graphs import layout_nodes
from pathlib import Path

In [ ]:
hv.extension("bokeh")

In [ ]:
data_path = Path("../src/day_20/data.txt")
data = data_path.read_text().splitlines()

In [ ]:
def parse_simple(line):
    name, dep = line.split(" -> ")
    dep = dep.split(", ")
    return {"name": name[1:], "op": name[0], "dep": dep}

In [ ]:
def parse_broadcaster(line):
    assert line.startswith("broadcaster")
    _, dep = line.split(" -> ")
    dep = dep.split(", ")
    return {"name": "broad", "op": "b", "dep": dep}

In [ ]:
def parse_line(line):
    if line[0] == "%" or line[0] == "&":
        return parse_simple(line)
    return parse_broadcaster(line)

In [ ]:
nodes = [parse_line(line) for line in data]

In [ ]:
op_names = {node["name"]: node["op"] + node["name"] for node in nodes}

In [ ]:
edges = []
for node in nodes:
    for d in node["dep"]:
        s = op_names.get(node["name"], node["name"])
        d = op_names.get(d, d)
        if d.startswith("&"):
            w = 2.0
        else:
            w = 1.0
        edges.append((s, d, w))

In [ ]:
graph = hv.Graph(edges, vdims="weight").opts(width=800, height=600, directed=True, edge_color="weight")
layout_nodes(graph, layout=nx.layout.fruchterman_reingold_layout, kwargs={'weight': 'weight'})